In [1]:
from __future__ import print_function    # (at top of module)

import os
import json
import time
import sys
from dotenv import load_dotenv

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

load_dotenv()
client_id = os.getenv("SPOTIPY_CLIENT_ID")
client_secret = os.getenv("SPOTIPY_CLIENT_SECRET")

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

In [2]:
urn = 'spotify:track:2jQiSYrwJehQAcuaaQrXnS'

sp.audio_features(urn)

[{'danceability': 0.422,
  'energy': 0.609,
  'key': 11,
  'loudness': -6.361,
  'mode': 0,
  'speechiness': 0.0335,
  'acousticness': 0.0114,
  'instrumentalness': 4.76e-06,
  'liveness': 0.108,
  'valence': 0.0911,
  'tempo': 107.34,
  'type': 'audio_features',
  'id': '2jQiSYrwJehQAcuaaQrXnS',
  'uri': 'spotify:track:2jQiSYrwJehQAcuaaQrXnS',
  'track_href': 'https://api.spotify.com/v1/tracks/2jQiSYrwJehQAcuaaQrXnS',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2jQiSYrwJehQAcuaaQrXnS',
  'duration_ms': 230011,
  'time_signature': 3}]